In [1]:
#hugging face packages
!pip install transformers datasets


     |████████████████████████████████| 1.8MB 12.3MB/s 
     |████████████████████████████████| 163kB 50.9MB/s 
     |████████████████████████████████| 890kB 48.2MB/s 
     |████████████████████████████████| 2.9MB 49.1MB/s 
     |████████████████████████████████| 17.7MB 210kB/s 
     |████████████████████████████████| 245kB 53.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=af74870ccdfa3e09721478940a5643086eaf477e427342bd24a5d8b5268f7c55
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1


In [2]:
#imports for the task
import io
import torch
from sklearn.model_selection import train_test_split
from transformers import pipeline
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import AutoModelForMaskedLM
from transformers import DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from transformers import AutoModelWithLMHead


In [ ]:
#This function creates a dataset using datasets library of hugging face
#The Eurparl eng data is divided into train,test and validation
def create_datasets(input_path,output_dir):
  lines = io.open(input_path).read().strip().split('\n')
  train, test = train_test_split(lines, test_size = 0.1)
  train, val = train_test_split(train, test_size = 0.05)
  with open(output_dir+"train", "w") as outfile:
      outfile.write("\n".join(train))
  with open(output_dir+"test", "w") as outfile:
      outfile.write("\n".join(test))
  with open(output_dir+"val", "w") as outfile:
      outfile.write("\n".join(val))
  datasets = load_dataset('text',data_files={'train': [output_dir+'train'], 'validation':[output_dir+'val'],'test': [output_dir+'test']})
  return datasets

In [ ]:
input_path='/content/drive/MyDrive/europarl-v7.fr-en.en' #path to dataset
output_dir='/content/'
datasets=create_datasets(input_path,output_dir)

Using custom data configuration default


Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-29540032ec8e59de/0.0.0/daf90a707a433ac193b369c8cc1772139bb6cca21a9c7fe83bdd16aad9b9b6ab. Subsequent calls will reuse this data.


In [ ]:
datasets #All the splits of the data can be seen below

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 1716602
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 90348
    })
    test: Dataset({
        features: ['text'],
        num_rows: 200773
    })
})

In [ ]:
model_checkpoint = "bert-base-uncased" 
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True) #using tokenizer of pretrained bert

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"])
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"]) #tokenizing each data split


Token indices sequence length is longer than the specified maximum sequence length for this model (576 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1099 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (670 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (519 > 512). Running this sequence through the model will result in indexing errors


Token indices sequence length is longer than the specified maximum sequence length for this model (627 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
block_size = 128 #input dimensions

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    total_length = (total_length // block_size) * block_size
    # Split by chunks of block_size.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

In [ ]:
#making use of gpu for model training
device = 'cpu'
if torch.cuda.is_available():
  device = 'cuda'

In [ ]:
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint) #using predefined architecture for training LM
model=model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15) #masking 15% of the inputs randomly 

In [ ]:
#Hyperparameters for our model
training_args = TrainingArguments(
    "/content/drive/MyDrive/FineTuneLM",#output directory
    evaluation_strategy = "epoch", #error calculated for every epoch
    learning_rate=2e-5,
    weight_decay=0.01,
    num_train_epochs=1.0 #training only for one epoch due to lack of resources
   
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
    data_collator=data_collator,
    
)

In [ ]:

trainer.train() #Lack of resources causes the trainer to collapse without completing trainig for a single epoch

Epoch,Training Loss,Validation Loss


RuntimeError: ignored

Since the training was stopped before fine tuning on the whole set( the data set is huge for colab to handle), I've exploited the partially fine tuned model using checkpoints.The inference step uses the partially fine tuned model to find similar words.

In [ ]:
#saving the trainer and tokenizer to drive
#trainer.save_model("/content/drive/MyDrive/FineTuneLM")
#tokenizer.save_pretrained("/content/drive/MyDrive/FineTuneLM")

In [18]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

#loading fine tuned model
model= AutoModelForMaskedLM.from_pretrained("/content/drive/MyDrive/FineTuneLM/checkpoint-1000") 


In [7]:
model= AutoModelForMaskedLM.from_pretrained("bert-base-uncased") #check results without fine tuning


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [29]:
#Fill Mask pipeline provided by hugging face
unmasker = pipeline('fill-mask', model=model,tokenizer= tokenizer)
unmasker("A product [MASK] is the marketing copy that explains what a product is and why it’s worth purchasing.")

[{'score': 0.11881297081708908,
  'sequence': '[CLS] a product copy is the marketing copy that explains what a product is and why it ’ s worth purchasing. [SEP]',
  'token': 6100,
  'token_str': 'copy'},
 {'score': 0.04442954435944557,
  'sequence': '[CLS] a product statement is the marketing copy that explains what a product is and why it ’ s worth purchasing. [SEP]',
  'token': 4861,
  'token_str': 'statement'},
 {'score': 0.03537537902593613,
  'sequence': '[CLS] a product reference is the marketing copy that explains what a product is and why it ’ s worth purchasing. [SEP]',
  'token': 4431,
  'token_str': 'reference'},
 {'score': 0.03226691111922264,
  'sequence': '[CLS] a product directive is the marketing copy that explains what a product is and why it ’ s worth purchasing. [SEP]',
  'token': 16449,
  'token_str': 'directive'},
 {'score': 0.02549256756901741,
  'sequence': '[CLS] a product document is the marketing copy that explains what a product is and why it ’ s worth purcha

In [9]:


#Code for fill mask pipeline
sequence = f"A product {tokenizer.mask_token} is the marketing copy that explains what a product is and why it’s worth purchasing."
# f"One of the people assassinated very recently in Sri Lanka was Mr Kumar Ponnambalam, who had visited the {tokenizer.mask_token} Parliament just a few months ago."

input = tokenizer.encode(sequence, return_tensors="pt") #tokenizing the give sequence
mask_token_index = torch.where(input == tokenizer.mask_token_id)[1]

token_logits = model(input)[0]
mask_token_logits = token_logits[0, mask_token_index, :]

top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()

for token in top_5_tokens:
    print(sequence.replace(tokenizer.mask_token, tokenizer.decode([token])))

A product guide is the marketing copy that explains what a product is and why it’s worth purchasing.
A product statement is the marketing copy that explains what a product is and why it’s worth purchasing.
A product copy is the marketing copy that explains what a product is and why it’s worth purchasing.
A product reference is the marketing copy that explains what a product is and why it’s worth purchasing.
A product concept is the marketing copy that explains what a product is and why it’s worth purchasing.
